In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np

C:\Users\Yash Thakar\AppData\Local\Temp\ipykernel_15420\3230018081.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_x = pd.read_csv("pre_processed_data/df_x_n_mfcc13.csv") 

In [3]:
df_y = pd.read_csv("pre_processed_data/df_y_n_mfcc13.csv")

In [4]:
x = np.array(df_x).reshape(-1, 13, 1)

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_y)

c:\Users\Yash Thakar\PROGRAMMING\Quantum\env\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
unique_elements, counts_elements = np.unique(y_encoded, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[  0   1   2   3   4]
 [174 229 210 222 138]]


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, 
                                                    test_size=0.2, random_state = 42)

In [8]:
x_train.shape

(778, 13, 1)

In [16]:
x_test.shape

(195, 13, 1)

In [11]:
# Create the model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(13, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax')) 


# Compile the model
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0002)
cb = [EarlyStopping(patience=10,monitor='accuracy',mode='max',restore_best_weights=True)]
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_test, y_test), callbacks = cb)

c:\Users\Yash Thakar\PROGRAMMING\Quantum\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.2058 - loss: 18.9351 - val_accuracy: 0.3179 - val_loss: 2.2802
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2433 - loss: 6.1184 - val_accuracy: 0.3282 - val_loss: 1.5025
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2079 - loss: 3.5549 - val_accuracy: 0.3179 - val_loss: 1.5024
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2391 - loss: 2.3188 - val_accuracy: 0.4205 - val_loss: 1.4246
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2882 - loss: 1.8147 - val_accuracy: 0.4410 - val_loss: 1.3864
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3062 - loss: 1.6214 - val_accuracy: 0.4923 - val_loss: 1.3422
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3486 - loss: 1.5110 - val_accuracy: 0.4308 - val_loss: 1.3243
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3491 - loss: 1.4859 - val_accuracy: 0.5077 -

In [12]:
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [13]:
y_pred.shape

(195, 5)

In [14]:
len(y_pred_classes)

195

In [28]:
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.5794871794871795


In [29]:
class_report = classification_report(y_test, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.21      0.33        33
           1       0.98      0.80      0.88        56
           2       0.34      0.81      0.48        37
           3       0.58      0.64      0.61        45
           4       1.00      0.08      0.15        24

    accuracy                           0.58       195
   macro avg       0.74      0.51      0.49       195
weighted avg       0.73      0.58      0.56       195



In [30]:
y_pred = model.predict(x_train)
y_pred_classes = y_pred.argmax(axis=1)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [31]:
accuracy = accuracy_score(y_train, y_pred_classes)
print(f'Accuracy Score: {accuracy}')

Accuracy Score: 0.5655526992287918


In [32]:
class_report = classification_report(y_train, y_pred_classes)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.29      0.43       141
           1       0.98      0.75      0.85       173
           2       0.40      0.84      0.54       173
           3       0.52      0.66      0.58       177
           4       1.00      0.05      0.10       114

    accuracy                           0.57       778
   macro avg       0.74      0.52      0.50       778
weighted avg       0.72      0.57      0.54       778

